# Algoritmo RSA

# Problema

El algoritmo RSA fue creado por Rivest, Shamir y Adleman.

El problema matemático en que se basa RSA es la factorización de números primos.

# Generación de claves

1. Se eligen dos números primos distintos $p$ y $q$

In [2]:
import random as r 

print("Calculando primos p y q ...")
p = 31
q = 127
print("p y q calculados.")
strp = str(p)
strq = str(q)
print ("p: " + strp)
print("q: " + strq)

Calculando primos p y q ...
p y q calculados.
p: 31
q: 127


2. Se calcula $n = p \cdot q$

In [3]:
print("Calculando n...")
n=p*q
print("n calculado.")
strn = str(n)
print("n: " + strn)

Calculando n...
n calculado.
n: 3937


3. Con la función de euler se calcula $\phi (n) = (p-1) \cdot (c-1)$

In [4]:
print("Calculando phi(n)...")
phi_n = (p-1) * (q-1)
print("phi(n) calculado.")
strphi_n = str(phi_n)
print("phi(n): " + strphi_n)

Calculando phi(n)...
phi(n) calculado.
phi(n): 3780


4. Se escoge un $e < \phi (n)$, que sea coprimo con $\phi (n)$

In [5]:
def mcd(a, b): #Máximo común divisor
    resto = 0
    while(b > 0):
        resto = b
        b = a % b
        a = resto
    return a

def hallarCoprimo(a, b): #Si mcd(A,B) = 1 entonces son coprimos
    maxcd = mcd(a,b)
    if(maxcd == 1):
        return a
    else:
        a = a+1
        return hallarCoprimo(a, b)

e = r.randint(2,(phi_n-1))
print("Hallando e coprimo de phi(n)...")
e = hallarCoprimo(e, phi_n)
stre = str(e)
print("e: " + stre)
prueba = mcd(e,phi_n)
print("Verificando que son coprimos...")
print("MCD(e,phi(n))= " + str(prueba))

Hallando e coprimo de phi(n)...
e: 2099
Verificando que son coprimos...
MCD(e,phi(n))= 1


5. Se determina un $d$ que satisfaga la congruencia $e\cdot d \equiv 1$ $(mod \phi (n))$

In [6]:
def resolverCongruencia(a, b, m): #ax \equiv b (mod m)
    if b == 0:
        return 0
    if a < 0:
        a = -a
        b = -b
    b %= m
    while a > m:
        a -= m
    return (m * resolverCongruencia(m, -b, a) + b) // a

print("Hallando d resolviendo la congruencia...")
d = resolverCongruencia(e,1,phi_n)
print(str(d) + " Satisface la congruencia")

Hallando d resolviendo la congruencia...
2939 Satisface la congruencia


La **clave pública** es $(n,e)$. La **clave privada** es $(n,d)$

In [7]:
print("Clave pública: (" + str(n) + ", " + str(e) + ")" )
print("Clave privada: (" + str(n) + ", " + str(d) + ")" )

Clave pública: (3937, 2099)
Clave privada: (3937, 2939)


# Cifrado

Para cifrar un mensaje $M$. Primero se convierte $M$ en un entero $m$, (con un protocolo arcodado y reversible) menor que $n$. Luego se computa el mensaje cifrado $c$ mediante la operación:
    
$c \equiv m^{e}$ $(mod$ $n)$    

In [8]:
m = 321
print("Mensaje: " + str(m))

Mensaje: 321


In [9]:
print("Cifrando " + str(m))
c = int(m) ** e % n
print("Cifrado: " + str(c))

Cifrando 321
Cifrado: 2745


# Descifrado

Para recuperar el mensaje $m$ a partir de $c$ usando su exponente $d$ de la clave privada se realiza el siguiente cálculo:

$m \equiv c^{d}$ $(mod$ $n)$

In [10]:
descifrado = c**d % n
print("Descifrado: " + str(descifrado))

Descifrado: 321


Ahora con $m$ puede convertirlo a $M$

# Implementación

In [11]:
#Importando librerías
import random as r

In [18]:
#Definiendo funciones
def mcd(a, b): #Máximo común divisor
    resto = 0
    while(b > 0):
        resto = b
        b = a % b
        a = resto
    return a

def hallarCoprimo(a, b): #Si mcd(A,B) = 1 entonces son coprimos
    maxcd = mcd(a,b)
    if(maxcd == 1):
        return a
    else:
        a = a + 1
        return hallarCoprimo(a, b)
    
def resolverCongruencia(a, b, m): #ax \equiv b (mod m)
    if b == 0:
        return 0
    if a < 0:
        a = -a
        b = -b
    b %= m
    while a > m:
        a -= m
    return (m * resolverCongruencia(m, -b, a) + b) // a

def powerModInt(a,k,n):
    b = 1
    if k == 0:
        return b
 
    while k>=1:
 
        if k&1:
            b = (b*a)%n
        a =(a*a)%n
        k=k>>1
 
    return b

def esPrimo(p): #Test primalidad Miller Rabin
    #Primero comprobamos que sea impar.
    if 1&p==0:
        return False
    #Expresamos p-1 como 2^u*s, con s impar.
    s = p-1
    #Dividir s por 2 hasta que el resultado sea impar.
    u = 0
    while 1&s==0:
        u= u+1
        s = s >> 1
    print(p,"= 2^",u,"*",s)
    for _ in range(20):#Ejecuciones para reducir las probabiliades de fallo.
        #Elegimos a al azar tal que 2 <= a <= p-2
        a = r.randint(2, p - 2)
        a = powerModInt(a,s,p)
        if a == 1 or a == p-1:#p-1 = -1
            return True
        else:
            for i in [1,1,u-1]:
                a = powerModInt(a,2,p)
                if a == p-1:
                    return True
                elif a == 1:
                    return False
                i=i+1
            return False
        
def encrypt(m):
    print("\nEncriptando mensaje...")
    c = m ** e % n
    return c

def decrypt(c):
    print("\nDesencriptando mensaje...")
    m = c ** d % n
    return m

In [22]:
#Eligiendo p y q
print("Calculando p y q...")

p = 401
q = 1567

strp = str(p)
strq = str(q)

print("p: " + strp)
print("q: " + strq)

#Calculando n = p*q
print("\nCalculando n...")

n = p * q
strn = str(n)
print("n: " + strn)

#Calculando phi(n)
print("Calculando phi(n)...")

phi_n = (p-1) * (q-1)

strphi_n = str(phi_n)
print("phi(n): " + strphi_n)

#Hallando e < phi(n)
print("\nHallando e coprimo de phi(n)...")

e = r.randint(2,(phi_n-1))
e = hallarCoprimo(e, phi_n)

stre = str(e)
print("e: " + stre)

prueba = mcd(e,phi_n)
print("\nVerificando que son coprimos...")
print("MCD(e,phi(n))= " + str(prueba))

#Hallando d que resuelve la congruencia e*d = 1 (mod phi(n))
print("\nHallando d resolviendo la congruencia...")

d = resolverCongruencia(e,1,phi_n)

strd = str(d)
print("d: " + strd + " satisface la congruencia")

#Imprimiendo clave pública y privada
print("\nClave pública: (" + strn + ", " + stre + ")" )
print("Clave privada: (" + strn + ", " + strd + ")" )


Calculando p y q...
p: 401
q: 1567

Calculando n...
n: 628367
Calculando phi(n)...
phi(n): 626400

Hallando e coprimo de phi(n)...
e: 387283

Verificando que son coprimos...
MCD(e,phi(n))= 1

Hallando d resolviendo la congruencia...
d: 171547 satisface la congruencia

Clave pública: (628367, 387283)
Clave privada: (628367, 171547)


In [23]:
##Encriptar
m = input("\nInserte mensaje m: ")
c = encrypt(int(m))
strc = str(c)
print("mensaje encriptado c: " + strc)




Inserte mensaje m: 24

Encriptando mensaje...
mensaje encriptado c: 99640


In [24]:
##Desencriptar
c = input("\nInserte mensaje encriptado c: ")
m = decrypt(int(c))
strm = str(m)
print("Mensaje desencriptado m: " + strm)


Inserte mensaje encriptado c: 99640

Desencriptando mensaje...
Mensaje desencriptado m: 24
